In [ ]:
!pip install boto3

Check if instance is Present....

In [ ]:
import boto3
ec2 = boto3.client("ec2")

response = ec2.describe_instances()

## instance_name = "cmorrismlops"

instance_name = "sentiment-analysis-mlops"

instance_id = ""

for resp in response["Reservations"]:
    resp = resp["Instances"][0]
    tags = resp.get("Tags",[])

    for tag in tags:
        if tag["Key", ""]== "Name" and tag.get("Value") == instance_name:
            instance_id = resp["InstanceId"]

if instance_id =="":
    print(f"There is no instace found with the name you gave me: {instance_name}")
    ## raise(STOP HERE!!)

instance_id





## Create an Amazon EC2 instance

In [ ]:
import boto3
ec2 = boto3.client("ec2")

if instance_id == "":
    